In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

# Data beans video feed generation

This notebook generate synthetic videos for the truck video feed.
 - Iterates over the `COND_IMAGES_DIR`, create the videos and put the resulting file on the `VIDEO_GEN_PATH` folder.
 - This notebook requires `CUDA` (GPU)

In [ ]:
! pip install diffusers
! pip install transformers
! pip install accelerate
! pip3 install torch torchvision torchaudio
! pip3 install opencv-python

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import torch
import os
import shutil
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video
from pathlib import Path

In [ ]:
#Ensure GPU is available
assert torch.cuda.is_available()
torch.cuda.empty_cache()

In [ ]:
COND_IMAGES_DIR = "images"
VIDEO_GEN_PATH = "videos"
os.makedirs(COND_IMAGES_DIR, exist_ok=True)
os.makedirs(VIDEO_GEN_PATH, exist_ok=True)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
for filename in uploaded.keys():
  shutil.copy(filename, COND_IMAGES_DIR)

In [ ]:
def _gen_video(cond_image, video_path):
    pipe = StableVideoDiffusionPipeline.from_pretrained("stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16")
    pipe.enable_model_cpu_offload()
    pipe.unet.enable_forward_chunking()

    image = load_image(cond_image)
    image = image.resize((476, 476))

    generator = torch.manual_seed(42)
    frames = pipe(image, decode_chunk_size=8, generator=generator).frames[0]

    export_to_video(frames, video_path, fps=25)


In [ ]:
def generate_videos():
  for image in os.listdir(COND_IMAGES_DIR):
      if not image.startswith("."):
          image_filename = os.path.join(COND_IMAGES_DIR,image)
          video_filename =  os.path.join(VIDEO_GEN_PATH,os.path.splitext(image)[0] + ".mp4")
          _gen_video(image_filename,video_filename)

In [ ]:
generate_videos()